<a href="https://colab.research.google.com/github/vnandigam/CHM_lidar_lidR/blob/main/OT_STAC_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OT Raster STAC Simple Example
This notebook demonstrates how to connect to the OpenTopography Raster STAC catalog to search for digital elevation model (DEM) datasets within a specified geographic bounding box. It retrieves available collections in the catalog, filters items whose bounding boxes intersect with the area of interest.

The notebook includes a function to generate hillshade images from elevation data, which are then used to create semi-transparent shaded relief overlays on an interactive map centered on the query area. This enables visualization of terrain features from the selected raster DEM assets. The code also handles geospatial metadata and coordinate reference system details for accurate rendering and pixel scaling.

Authors: Minh Phan, Viswanath Nandigam

info@opentopography.org

In [ ]:
# Import core libraries: STAC Python Client, Array math (NumPy), Raster I/O (rasterio), Interactive web map display (ipyleaflet Map and ImageOverlay). in‑memory image creation and encoding (Pillow, io, base64).

!pip install --quiet pystac-client rasterio

import json
from pystac_client import Client
import numpy as np
import rasterio
from ipyleaflet import Map, ImageOverlay, basemaps
from IPython.display import display
from PIL import Image
import io, base64

In [ ]:
# Connect to the OpenTopography Raster STAC catalog
URL = 'https://twsa.ucsd.edu/stac/raster_catalog.json'
client = Client.from_file(URL)

# Define the spatial bounding box of interest as [min_lon, min_lat, max_lon, max_lat]
bbox = [-119.6, 37.7, -119.5, 37.8]

# Retrieve collections available in the STAC catalog
collections = client.get_collections()

# Start search and iterate through collections and items
item_count = 0

print("Searching for items...")

for collection in collections:
    # print(f"\nCollection: {collection.title}")

    # Get items from this collection
    items = collection.get_items()

    for item in items:
        # Check if item intersects with bounding box
        if item.bbox:
            item_bbox = item.bbox
            # Intersection logic
            if not (item_bbox[2] < bbox[0] or  # item east < search west
                    item_bbox[0] > bbox[2] or  # item west > search east
                    item_bbox[3] < bbox[1] or  # item north < search south
                    item_bbox[1] > bbox[3]):   # item south > search north

                print(f"\nCollection Title: {collection.title}")
                print(f"Collection ID: {collection.id}")
                print(f"Item ID: {item.id}")
                item_count += 1

                # break after 5 intersecting items found
                if item_count >= 5:
                    break

    if item_count >= 5:
        break


In [ ]:
# Retrieve a metadata for the specific data Collection using the collection ID
# e.g. ALOS World 3D - 30m

collection = client.get_collection("OTALOS.112016.4326.2")

# Print all metadata in JSON
collection_dict = collection.to_dict()
print(json.dumps(collection_dict, indent=2))

In [ ]:
# Check if two bounding boxes intersect (minx, miny, maxx, maxy)
def intersects(a, b):
    return not (a[2] <= b[0] or a[0] >= b[2] or a[3] <= b[1] or a[1] >= b[3])

"""
Generate hillshade (shaded relief)
Parameters:
        arr (np.ndarray): 2D array of elevation values.
        azimuth (float): Sun azimuth in degrees (default 315° = NW).
        altitude (float): Sun altitude in degrees above horizon (default 45°).
        dx (float): Spacing between cells in x-direction (default 1).
        dy (float): Spacing between cells in y-direction (default 1).
"""

def hillshade(arr, azimuth=315.0, altitude=45.0, dx=1.0, dy=1.0):
    # Handle NaN values, fill with mean or zero if mean is NaN
    if np.isnan(arr).any():
        fill = np.nanmean(arr)
        if np.isnan(fill):
            arr = np.where(np.isnan(arr), 0, arr)
        else:
            arr = np.where(np.isnan(arr), fill, arr)

    # Compute gradients in y and x directions
    gy, gx = np.gradient(arr, dy, dx)

    # Calculate slope and aspect
    slope = np.pi/2.0 - np.arctan(np.hypot(gx, gy))
    aspect = np.arctan2(-gx, gy)

    # Convert sun azimuth and altitude to radians
    az = np.deg2rad(azimuth)
    alt = np.deg2rad(altitude)

    # Calculate hillshade based on illumination model
    hs = np.sin(alt) * np.sin(slope) + np.cos(alt) * np.cos(slope) * np.cos(az - aspect)

    # Clamp values between 0 and 1 for valid shading
    np.clip(hs, 0, 1, out=hs)

    # Scale to 0-255 for image representation
    return (hs * 255).astype(np.uint8)


# Find the first item in the collection whose bounding box intersects the query bounding box
first_item = next((it for it in collection.get_items() if it.bbox and intersects(it.bbox, bbox)), None)

if first_item is None:
    print("No item intersects the search bbox.")
else:
    # Collect GeoTIFF-like assets whose bbox intersects the query bbox
    chosen_assets = []
    for asset in first_item.assets.values():
        mt = (getattr(asset, "media_type", "") or getattr(asset, "type", "")).lower()
        if "tiff" in mt:
            asset_bbox = getattr(asset, "extra_fields", {}).get("bbox") or first_item.bbox
            if asset_bbox and intersects(asset_bbox, bbox):
                chosen_assets.append((asset, asset_bbox))

    if not chosen_assets:
        print("No TIFF assets intersect the search bbox.")
    else:
        # Extract query bbox coordinates
        west_q, south_q, east_q, north_q = bbox

        # Initialize interactive map centered on query bbox midpoint with zoom level 8
        m = Map(center=((south_q + north_q) / 2.0, (west_q + east_q) / 2.0), zoom=8, scroll_wheel_zoom=True)
        m.layout.height = "600px"

        # Loop over chosen GeoTIFF assets to create hillshade overlays on the map
        for idx, (asset, asset_bbox) in enumerate(chosen_assets, start=1):
            west, south, east, north = asset_bbox

            with rasterio.open(asset.href) as ds:
                band = ds.read(1).astype("float32")
                if ds.nodata is not None:
                    band = np.where(band == ds.nodata, np.nan, band)

                # Approximate pixel size in meters for gradient calculation
                if ds.crs and ds.crs.is_geographic:
                    # Geographic CRS: convert degrees to meters based on latitude
                    mid_lat = (south + north) / 2.0
                    m_per_deg_lat = 111_132.0
                    m_per_deg_lon = 111_320.0 * np.cos(np.deg2rad(mid_lat))
                    dx = abs(ds.transform.a) * m_per_deg_lon
                    dy = abs(ds.transform.e) * m_per_deg_lat
                else:
                    # Projected CRS: pixel size is directly from transform
                    dx = abs(ds.transform.a)
                    dy = abs(ds.transform.e)

            # Generate hillshade image and alpha mask for valid data pixels
            hs = hillshade(band, dx=dx, dy=dy)
            alpha = (~np.isnan(band)).astype(np.uint8) * 255
            rgba = np.dstack([hs, hs, hs, alpha]).astype(np.uint8)

            # Encode RGBA image as PNG data URL for map overlay
            buf = io.BytesIO()
            Image.fromarray(rgba).save(buf, format="PNG", optimize=True)
            data_url = "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode("ascii")

            # Add image overlay to map with bounds and slight opacity
            m.add_layer(ImageOverlay(
                url=data_url,
                bounds=((south, west), (north, east)),
                opacity=0.95,
                name=f"Hillshade {idx}"
            ))

        display(m)